## Xente Fraud Detection
Competition : https://zindi.africa/competitions/xente-fraud-detection-challenge

Problem statement: Create a machine learning model to detect fraudulent transactions.

Predict `FraudResult` probability

Evaluation: The error metric for this competition is the `F1 score`, which ranges from 0 (total failure) to 1 (perfect score). Hence, the closer your score is to 1, the better your model.

In [1]:
# Load libraries
import numpy as np
import pandas as pd

### Data wrangling
This section focuses on cleaning and making the data usable for exploration and machine learning

**Combine Train and Test**
1. Combine Train and Test datasets to reduce number of steps

**Removing Extraneous Data**
1. Removing columns based on description that: 
    * Contained redundant information.
2. Removing columns with only one unique value.  


In [2]:
import warnings
warnings.simplefilter("ignore")

In [3]:
# Load datasets
df_test = pd.read_csv('../data/raw/test.csv')
df_train = pd.read_csv('../data/raw/training.csv')

In [4]:
# viewing the size of the dataset
print('The size of the train dataset: ' + str(df_train.shape))
print('The size of the test dataset: ' + str(df_test.shape))

The size of the train dataset: (95662, 16)
The size of the test dataset: (45019, 15)


In [5]:
# viewing the train dataset
df_train.head()

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult
0,TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,UGX,256,ProviderId_6,ProductId_10,airtime,ChannelId_3,1000.0,1000,2018-11-15T02:18:49Z,2,0
1,TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-20.0,20,2018-11-15T02:19:08Z,2,0
2,TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,UGX,256,ProviderId_6,ProductId_1,airtime,ChannelId_3,500.0,500,2018-11-15T02:44:21Z,2,0
3,TransactionId_380,BatchId_102363,AccountId_648,SubscriptionId_2185,CustomerId_988,UGX,256,ProviderId_1,ProductId_21,utility_bill,ChannelId_3,20000.0,21800,2018-11-15T03:32:55Z,2,0
4,TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-644.0,644,2018-11-15T03:34:21Z,2,0


In [6]:
# viewing the test dataset
df_test.head()

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy
0,TransactionId_50600,BatchId_35028,AccountId_2441,SubscriptionId_4426,CustomerId_2857,UGX,256,ProviderId_5,ProductId_3,airtime,ChannelId_3,1000.0,1000,2019-02-13T10:01:40Z,4
1,TransactionId_95109,BatchId_45139,AccountId_3439,SubscriptionId_2643,CustomerId_3874,UGX,256,ProviderId_5,ProductId_15,financial_services,ChannelId_3,2000.0,2000,2019-02-13T10:02:12Z,2
2,TransactionId_47357,BatchId_74887,AccountId_4841,SubscriptionId_3829,CustomerId_2857,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-50.0,50,2019-02-13T10:02:30Z,2
3,TransactionId_28185,BatchId_11025,AccountId_2685,SubscriptionId_4626,CustomerId_3105,UGX,256,ProviderId_5,ProductId_10,airtime,ChannelId_3,3000.0,3000,2019-02-13T10:02:38Z,4
4,TransactionId_22140,BatchId_29804,AccountId_4841,SubscriptionId_3829,CustomerId_3105,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-60.0,60,2019-02-13T10:02:58Z,2


From the above table outputs the train dataset has 95662 rows and 16 columns and the test dataset has 45019 rows and 16 columns. From initial observations made, a number of columns will not be useful for the fraud classification analysis.


**Combine Train and Test**

Combining Train and Test datasets will save time, feature engineering steps will not be done twice.
A column will be added to help identify both datasets.

In [7]:
# combining train and test datasets
df_test['source'] = 'test'
df_train['source'] = 'train'
df_data = pd.concat([df_train,df_test], ignore_index=True)

**Removing columns based on descriptions**

The columns containing information not useful for fraud transaction classification (e.g identifier columns) are removed. 


In [8]:
# listing names of columns in the dataset
df_data.columns

Index(['AccountId', 'Amount', 'BatchId', 'ChannelId', 'CountryCode',
       'CurrencyCode', 'CustomerId', 'FraudResult', 'PricingStrategy',
       'ProductCategory', 'ProductId', 'ProviderId', 'SubscriptionId',
       'TransactionId', 'TransactionStartTime', 'Value', 'source'],
      dtype='object')

Looking at the list above, the columns not needed for the intended analysis are listed in tables below. The first table below shows the columns that are removed because they contain information not needed.

|Count| Column title         | Description                                                                          |
|-----|----------------------|--------------------------------------------------------------------------------------|
|  1  |BatchId               |Unique number assigned to a batch of transactions for processing                      |
|  2  |AccountId             |Unique number identifying the customer on platform                                    |
|  3  |SubscriptionId        |Unique number identifying the customer subscription                                   |
|  4  |CustomerId            |Unique identifier attached to AccountId                                               |


In [9]:
# create a list of the columns to drop
# CustomerId will be dropped later, it is needed for transaction aggregation
columns_to_drop = ['BatchId','AccountId','SubscriptionId']

# drop the columns listed above  
df_data = df_data.drop(columns_to_drop, axis=1)

**Removing columns with one unique value** 

In [10]:
# printing the size of the dataset before removing
print('The size of the dataset before removing: ' + str(df_data.shape))

# removing columns with only one unique value 
df_data = df_data.loc[:,df_data.apply(func=pd.Series.nunique, args=(False)) > 1]

# printing the size of the dataset
print('The size of the dataset after removing: ' + str(df_data.shape))

The size of the dataset before removing: (140681, 14)
The size of the dataset after removing: (140681, 12)


The names and description of the remaining 11 columns are show below

In [11]:
# importing the xente fraud data dictionary
df_xente_dictionary = pd.read_csv('../data/raw/Xente_Variable_Definitions.csv', index_col = 0)

In [12]:
# ensuring descriptions are not truncated
pd.options.display.max_colwidth =300

# adding descriptions for newly added column
df_xente_dictionary.loc['source'] = 'The source of the record, train or test datasets'

# displaying the descriptions of these columns 
df_xente_dictionary.loc[df_data.columns.tolist(), :]

,Definition
Column Name,
Amount,Value of the transaction. Positive for debits from customer account and negative for credit into customer account
ChannelId,"Identifies if customer used web,Android, IOS, pay later or checkout."
CustomerId,Unique identifier attached to Account
FraudResult,Fraud status of transaction 1 -yes or 0-No
PricingStrategy,Category of Xente's pricing structure for merchants
ProductCategory,ProductIds are organized into these broader product categories.
ProductId,Item name being bought.
ProviderId,Source provider of Item �bought.
TransactionId,Unique transaction identifier on platform


In [13]:
# resetting the max column width display 
pd.options.display.max_colwidth = 50

In [14]:
df_data.head()

,Amount,ChannelId,CustomerId,FraudResult,PricingStrategy,ProductCategory,ProductId,ProviderId,TransactionId,TransactionStartTime,Value,source
0,1000.0,ChannelId_3,CustomerId_4406,0.0,2,airtime,ProductId_10,ProviderId_6,TransactionId_76871,2018-11-15T02:18:49Z,1000,train
1,-20.0,ChannelId_2,CustomerId_4406,0.0,2,financial_services,ProductId_6,ProviderId_4,TransactionId_73770,2018-11-15T02:19:08Z,20,train
2,500.0,ChannelId_3,CustomerId_4683,0.0,2,airtime,ProductId_1,ProviderId_6,TransactionId_26203,2018-11-15T02:44:21Z,500,train
3,20000.0,ChannelId_3,CustomerId_988,0.0,2,utility_bill,ProductId_21,ProviderId_1,TransactionId_380,2018-11-15T03:32:55Z,21800,train
4,-644.0,ChannelId_2,CustomerId_988,0.0,2,financial_services,ProductId_6,ProviderId_4,TransactionId_28195,2018-11-15T03:34:21Z,644,train


In [16]:
# export data for exploration and machine learning 
df_data.to_csv('../data/interim/wrangled_data.csv', index = False)